In [1]:
import numpy as np
import pandas as pd
from os.path import join
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision import models
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

# Dataset class
class AgeDataset(Dataset):
    def __init__(self, data_path, annot_path, train=True):
        super(AgeDataset, self).__init__()
        self.data_path = data_path
        self.train = train
        self.ann = pd.read_csv(annot_path)
        self.files = self.ann['file_id']
        if train:
            self.ages = self.ann['age']
        self.transform = self._transform(224)

    @staticmethod
    def _convert_image_to_rgb(image):
        return image.convert("RGB")

    def _transform(self, n_px):
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        return Compose([
            Resize(n_px),
            self._convert_image_to_rgb,
            ToTensor(),
            Normalize(mean, std),
        ])

    def read_img(self, file_name):
        img = Image.open(join(self.data_path, file_name))
        return self.transform(img)

    def __getitem__(self, index):
        file_name = self.files[index]
        img = self.read_img(file_name)
        if self.train:
            age = self.ages[index]
            return img, age
        else:
            return img

    def __len__(self):
        return len(self.files)

# Paths
train_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train'
train_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv'
test_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test'
test_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv'

# Dataset and DataLoader
dataset = AgeDataset(train_path, train_ann, train=True)
train_dataset, val_dataset = random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
test_dataset = AgeDataset(test_path, test_ann, train=False)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Prediction function
@torch.no_grad
def predict(loader, model, device):
    model.eval()
    predictions = []
    for img in tqdm(loader):
        img = img.to(device)
        pred = model(img)
        predictions.extend(pred.cpu().flatten().tolist())
    return predictions

# Validation loop
def validation_loop(val_loader, model, criterion, device):
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for images, ages in tqdm(val_loader):
            images, ages = images.to(device), ages.to(device).float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, ages)
            running_val_loss += loss.item() * images.size(0)
    return running_val_loss / len(val_loader.dataset)

# Training loop
def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, device, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        train_running_loss = 0.0
        for images, ages in tqdm(train_loader):
            images, ages = images.to(device), ages.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, ages)
            loss.backward()
            optimizer.step()
            train_running_loss += loss.item() * images.size(0)

        epoch_loss = train_running_loss / len(train_loader.dataset)
        val_loss = validation_loop(val_loader, model, criterion, device)
        print(f"Epoch {epoch + 1}, Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}")

        preds = predict(test_loader, model, device)
        submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
        submit['age'] = preds
        submit.to_csv(f'/kaggle/working/submission_epoch_{epoch + 1}.csv', index=False)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model
model = models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

for name, param in model.named_parameters():
    if "layer1" in name:
        param.requires_grad = False

model = model.to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, device, num_epochs=40)

# Create submission CSV file
preds = predict(test_loader, model, device)
submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
submit['age'] = preds
submit.to_csv('/kaggle/working/submission.csv', index=False)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 90.3MB/s]
100%|██████████| 67/67 [00:09<00:00,  7.14it/s]


Epoch 1, Training Loss: 539.2432, Validation Loss: 275.6130


100%|██████████| 67/67 [00:05<00:00, 11.39it/s]


Epoch 2, Training Loss: 113.7068, Validation Loss: 71.8692


100%|██████████| 67/67 [00:06<00:00, 10.57it/s]


...

100%|██████████| 67/67 [00:05<00:00, 11.44it/s]


Epoch 39, Training Loss: 3.0177, Validation Loss: 49.0025


100%|██████████| 67/67 [00:06<00:00, 10.18it/s]


Epoch 40, Training Loss: 4.9771, Validation Loss: 53.3894


100%|██████████| 31/31 [00:02<00:00, 11.17it/s]
